# Homework: Introduction

In this homework, we'll learn more about search and use Elastic Search for practice. 

## Contents
- [Running Elastic](#q1)
- [Indexing the data](#q2)
- [Searching](#q3)
- [Filtering](#q4)
- [Building a prompt](#q5)
- [Tokens](#q6)

## Q1. Running Elastic<a id="q1"></a>

Run Elastic Search 8.4.3, and get the cluster information. If you run it on localhost, this is how you do it:

```bash
curl localhost:9200
```

What's the `version.build_hash` value?

In [1]:
!curl http://localhost:9200

{
  "name" : "4f0265ce67dd",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "GmC8WAl6SsGRahTbEPwOdQ",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


## Getting the data

Now let's get the FAQ data. You can run this snippet:

```python
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
```

Note that you need to have the `requests` library:

```bash
pip install requests
```

In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

## Q2. Indexing the data<a id="q2"></a>

Index the data in the same way as was shown in the course videos. Make the `course` field a keyword and the rest should be text. 

Don't forget to install the ElasticSearch client for Python:

```bash
pip install elasticsearch
```

Which function do you use for adding your data to elastic?

* `insert`
* `index`
* `put`
* `add`

In [3]:
from elasticsearch import Elasticsearch

es = Elasticsearch('http://localhost:9200')

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
        }
    }
}

index_name = 'course-questions'

es.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [4]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es.index(index=index_name, document=doc)

/home/colelandolt/workspace/llm-zoomcamp/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 948/948 [00:12<00:00, 75.57it/s]


In [5]:
print('index')

index


## Q3. Searching<a id="q3"></a>

Now let's search in our index. 

We will execute a query "How do I execute a command in a running docker container?". 

Use only `question` and `text` fields and give `question` a boost of 4, and use `"type": "best_fields"`.

What's the score for the top ranking result?

* 94.05
* 84.05
* 74.05
* 64.05

Look at the `_score` field.

In [6]:
query = "How do I execute a command in a running docker container?"

search_query = {
    "size": 1,
    "query": {
        "bool": {
            "must":{
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

response = es.search(index=index_name, body=search_query)

print(response['hits']['hits'][0]['_score'])

83.377396


## Q4. Filtering<a id="q4"></a>

Now let's only limit the questions to `machine-learning-zoomcamp`.

Return 3 results. What's the 3rd question returned by the search engine?

* How do I debug a docker container?
* How do I copy files from a different folder into docker container’s working directory?
* How do Lambda container images work?
* How can I annotate a graph?

In [7]:
def elastic_search(
    index_name: str,
    query: str,
    size: int = 5,
    filter: str = None
) -> dict:
    # Create the search query
    search_query = {
        "size": size,
        "query": {
            "bool": {
                "must":{
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

    # Filter by course if needed
    if filter is not None:
        search_query['query']['bool']["filter"] = {
            "term": {
                "course": filter
            }
        }

    # Search the index
    response = es.search(index=index_name, body=search_query)

    # Extract the search results
    search_results = []

    for hit in response['hits']['hits']:
        search_results.append(hit['_source'])

    # Return the search results
    return search_results

search_results = elastic_search(
    index_name=index_name,
    query=query,
    size=3,
    filter='machine-learning-zoomcamp'
)
print(search_results[2]['question'])

How do I copy files from a different folder into docker container’s working directory?


## Q5. Building a prompt<a id="q5"></a>

Now we're ready to build a prompt to send to an LLM. 

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (`\n\n`)
```python
context_template = """
Q: {question}
A: {text}
""".strip()
```

Now use the context you just created along with the "How do I execute a command in a running docker container?" question 
to construct a prompt using the template below:

```
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
```

What's the length of the resulting prompt? (use the `len` function)

* 962
* 1462
* 1962
* 2462

In [8]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teach assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the question,
If the CONTEXT doesn't contain the answer output NONE

QUESTION: {question}

CONTEXT: 
{context}
""".strip()
    
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    return prompt_template.format(question=query, context=context)

prompt = build_prompt(query, search_results)

print(len(prompt))

1690


## Q6. Tokens<a id="q6"></a>

When we use the OpenAI Platform, we're charged by the number of 
tokens we send in our prompt and receive in the response.

The OpenAI python package uses `tiktoken` for tokenization:

```bash
pip install tiktoken
```

Let's calculate the number of tokens in our query: 

```python
encoding = tiktoken.encoding_for_model("gpt-4o")
```

Use the `encode` function. How many tokens does our prompt have?

* 122
* 222
* 322
* 422

Note: to decode back a token into a word, you can use the `decode_single_token_bytes` function:

```python
encoding.decode_single_token_bytes(63842)
```

In [9]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")

print(len(encoding.encode(prompt)))

368
